In [1]:
import dask
from dask.distributed import Client, progress
import dask.dataframe as dd
import numpy as np
import re
import dask_xgboost as dxgb
from dask_ml.model_selection import train_test_split as dd_split
import pandas as pd
from sklearn.model_selection import train_test_split as pd_split

C:\Users\8prab\.conda\envs\conda3.7.2\lib\site-packages\dask\array\random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,


In [2]:
dask.__version__

'2.11.0'

In [3]:
#client = Client(processes=False, threads_per_worker=4,
#                n_workers=1, memory_limit='2GB')
#client

## Creation of Dask Cluster

In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
cluster

In [5]:
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:62307 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.46 GB


## Data Preprocessing

In [6]:
train = dd.read_csv('train.csv')
train,val = dd_split(train,test_size=0.2,random_state=0)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
train.compute()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
3,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
4,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
...,...,...,...,...,...,...,...,...,...,...,...,...
689,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
690,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
691,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
692,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [7]:
test = dd.read_csv('test.csv')
test.compute()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [8]:
gender = dd.read_csv('gender_submission.csv')
gender.compute()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [9]:
PassengerId = test['PassengerId']
full_data = [train,val,test]

train['Name_length'] = train['Name'].apply(len)
val['Name_length'] = val['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)

train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
val['Has_Cabin'] = val["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

C:\Users\8prab\.conda\envs\conda3.7.2\lib\site-packages\dask\dataframe\core.py:3046: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Name', 'int64'))

  warnings.warn(meta_warning(meta))
C:\Users\8prab\.conda\envs\conda3.7.2\lib\site-packages\dask\dataframe\core.py:3046: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Cabin', 'i

In [10]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

In [11]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [12]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(np.median(train['Fare']))

In [13]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

C:\Users\8prab\.conda\envs\conda3.7.2\lib\site-packages\dask\dataframe\core.py:3046: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Name', 'object'))

  warnings.warn(meta_warning(meta))


In [14]:
def apply_title(val):
    if val in ['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']:
        return 'Rare'
    elif val in ['Mlle','Ms']:
        return 'Miss'
    elif val == 'Mme':
        return 'Mrs'
    else:
        return val

In [15]:
for dataset in full_data:
    dataset['Title'].apply(apply_title)

C:\Users\8prab\.conda\envs\conda3.7.2\lib\site-packages\dask\dataframe\core.py:3046: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Title', 'object'))

  warnings.warn(meta_warning(meta))


In [16]:
for dataset in full_data:
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [17]:
def apply_fare(val):
    if val <= 7.91:
        return 0
    elif val > 7.91 and val <= 14.454:
        return 1
    elif val > 14.454 and val <= 31:
        return 2
    elif val > 31:
        return 3

In [18]:
def apply_age(val):
    if val <= 16:
        return 0
    elif val > 16 and val <= 32:
        return 1
    elif val > 32 and val <= 48:
        return 2
    elif val > 48 and val <= 64:
        return 3
    elif val > 64:
        return 4
    else:
        return 5

In [19]:
def apply_single(val):
    if val['FamilySize'] > 1:
        return 0
    else:
        return 1

In [20]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].apply(apply_fare)
    dataset['Age'] = dataset['Age'].apply(apply_age)
    dataset['Single'] = dataset.apply(lambda x : apply_single(x),axis=1)

C:\Users\8prab\.conda\envs\conda3.7.2\lib\site-packages\dask\dataframe\core.py:3046: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Fare', 'int64'))

  warnings.warn(meta_warning(meta))
C:\Users\8prab\.conda\envs\conda3.7.2\lib\site-packages\dask\dataframe\core.py:3046: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Age', 'int

In [21]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
val = val.drop(drop_elements, axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [22]:
train.compute().shape

(694, 12)

In [23]:
val.compute().shape

(197, 12)

In [24]:
#train, validation = train.random_split([0.80, 0.20])

## XGBoost DASK

In [25]:
params = {'objective': 'binary:logistic',
          'max_depth': 64, 'eta': 0.01,
          'min_child_weight': 1, 'tree_method': 'hist',
          'grow_policy': 'lossguide'}

In [26]:
%%time
bst = dxgb.train(client, params, train.drop('Survived', axis = 1), train['Survived'])

Wall time: 2.27 s


In [27]:
#from dask_ml.xgboost import XGBClassifier
#est = XGBClassifier()
#est.fit(train.drop('Survived', axis = 1), train['Survived'])

In [28]:
predictions = dxgb.predict(client, bst, val.drop('Survived', axis = 1))

In [29]:
from sklearn.metrics import roc_auc_score, roc_curve,accuracy_score
print(roc_auc_score(val['Survived'].compute(),predictions.compute()))

0.8445033751205401


In [30]:
predictions1 = np.where(predictions > 0.5, 1, 0)

In [31]:
accuracy_score(val['Survived'].compute(), predictions1.compute())

0.817258883248731

## XGBoost non dask

In [32]:
import xgboost as xgb
val_pd = val.compute()
train_pd = train.compute()
test_pd = test.compute()

train_pd_xg = xgb.DMatrix(train_pd.drop('Survived',axis=1),label=train_pd['Survived'])
test_pd_xg = xgb.DMatrix(test_pd)
val_pd_xg = xgb.DMatrix(val_pd.drop('Survived',axis=1),label=val_pd['Survived'])

In [33]:
%%time
xgb_train = xgb.train(params, train_pd_xg)

Wall time: 23.5 ms


In [34]:
preds = xgb_train.predict(val_pd_xg)

In [35]:
print(roc_auc_score(val_pd['Survived'],preds))

0.8445033751205401


In [36]:
predictions2 = np.where(preds > 0.5, 1, 0)

In [37]:
accuracy_score(val_pd['Survived'],predictions2)

0.817258883248731

## Conclusion
Non Dask version has higher roc_auc score while accuracy on a threshold of 0.5 is same